In [15]:
import pandas as pd
import numpy as np
import wrds
import datetime as dt
from pandas.tseries.offsets import *
import matplotlib.pyplot as plt

conn=wrds.Connection() 

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


### TABLE 1. SUMMARY STATISTICS 

## Panel A - Percentage institutional ownership 

In [ ]:
# ORIGINAL 

crsp_m = conn.raw_sql("""
    SELECT a.permno, a.date, a.prc, a.shrout, a.cfacpr, a.cfacshr, exchcd 
    FROM crsp.msf AS a
    LEFT JOIN crsp.msenames AS b
    ON a.permno = b.permno
    AND b.namedt <= a.date
    AND a.date <= b.nameendt
    WHERE a.date BETWEEN '03/01/1983' AND '12/31/1997'  
    AND b.shrcd BETWEEN 10 AND 11
""", date_cols=['date'])

## FOR EXTENSION ONLY 

Preferred stock = 18 
ADRs = 31 

In [ ]:
# TABULATE 
shrcd_counts = crsp_m['shrcd'].value_counts().sort_index()
shrcd_pct = (shrcd_counts / shrcd_counts.sum() * 100).round(2)

shrcd_summary = pd.DataFrame({
    'Frequency': shrcd_counts,
    'Percentage': shrcd_pct
}).reset_index().rename(columns={'index': 'shrcd'})

# print(shrcd_summary)

shrcd_summary.to_excel("shrcd_summary.xlsx", index=False) 

In [36]:
crsp_m = conn.raw_sql("""
    SELECT a.permno, a.date, a.prc, a.shrout, a.cfacpr, a.cfacshr, b.exchcd, b.shrcd
    FROM crsp.msf AS a
    LEFT JOIN crsp.msenames AS b
    ON a.permno = b.permno
    AND b.namedt <= a.date
    AND a.date <= b.nameendt
    WHERE a.date BETWEEN '03/01/1983' AND '12/31/1997' 
    AND b.shrcd = 31 
""", date_cols=['date']) 

In [ ]:
# print(crsp_m['date'].min(), crsp_m['date'].max()) 

1983-03-31 00:00:00 2024-12-31 00:00:00


In [37]:
##### Only keep stocks on NYSE (1), AMEX (2), or NASDAQ (3) ##### 
crsp_m = crsp_m[crsp_m['exchcd'].isin([1, 2, 3])] 

In [38]:
# Adjust prices and compute market cap
crsp_m['p'] = crsp_m['prc'].abs() / crsp_m['cfacpr']
crsp_m['tso'] = crsp_m['shrout'] * crsp_m['cfacshr'] * 1e3  # shares outstanding
crsp_m['me'] = crsp_m['p'] * crsp_m['tso'] / 1e6  # market cap in $ millions

crsp_m['mdate'] = pd.to_datetime(crsp_m['date']) + pd.offsets.MonthEnd(0)
crsp_m['qdate'] = pd.to_datetime(crsp_m['date']) + pd.offsets.QuarterEnd(0)

# Keep only last month in each quarter 
qend = crsp_m[['permno', 'mdate', 'qdate']].groupby(['permno', 'qdate'])['mdate'].max().reset_index()
crsp_qend = pd.merge(crsp_m, qend, on=['permno', 'qdate', 'mdate'], how='inner')

# Collapse 
total_market_cap = crsp_qend.groupby('qdate')['me'].sum().reset_index(name='total_market_cap')

# Rename for merging
total_market_cap = total_market_cap.rename(columns={'qdate': 'fdate'})

# 13F Holdings 
s34type3 = conn.raw_sql("""
    SELECT fdate, mgrno, cusip, shares
    FROM tfn.s34type3
    WHERE fdate BETWEEN '03/01/1983' AND '12/31/1997'
""", date_cols=['fdate'])

# Institution Type 
s34names = conn.raw_sql("""
    SELECT DISTINCT mgrno, typecode
    FROM tfn.s34names 
""")

# CRSP linking 
crsp_link = conn.raw_sql("""
    SELECT DISTINCT permno, ncusip
    FROM crsp.msenames
    WHERE ncusip IS NOT NULL
""")

# Merge s34type3 with CRSP to get permno
s34type3 = pd.merge(s34type3, crsp_link, how='left', left_on='cusip', right_on='ncusip')
s34type3 = s34type3.drop(['cusip', 'ncusip'], axis=1)

# Merge with CRSP market cap data (on permno and fdate ≈ mdate)
s34type3['fdate'] = pd.to_datetime(s34type3['fdate'])
crsp_m['mdate'] = pd.to_datetime(crsp_m['mdate'])

holdings = pd.merge(
    s34type3,
    crsp_m[['permno', 'mdate', 'tso', 'me']],
    left_on=['permno', 'fdate'],
    right_on=['permno', 'mdate'],
    how='left'
) 

In [ ]:
# Step 1: Check for duplicate columns
# print("Columns in holdings:", holdings.columns)
# print("Columns in s34names:", s34names.columns)

Columns in holdings: Index(['fdate', 'mgrno', 'shares', 'permno', 'mdate', 'tso', 'me'], dtype='object')
Columns in s34names: Index(['mgrno', 'typecode'], dtype='object')


In [ ]:
# print(total_market_cap.columns)

Index(['fdate', 'total_market_cap'], dtype='object')


In [39]:
holdings = pd.merge(
    s34type3,
    s34names,
    on='mgrno',
    how='left'
)

crsp_tso = crsp_m[['permno', 'mdate', 'tso']].drop_duplicates()
holdings = pd.merge(
    holdings,
    crsp_tso,
    left_on=['permno', 'fdate'],
    right_on=['permno', 'mdate'],
    how='left'
)

holdings['typecode'] = holdings['typecode'].astype(int)  # ensure typecode is int

holdings['ownership_fraction'] = holdings['shares'] / holdings['tso']

# Drop rows with missing or infinite values
holdings = holdings.replace([np.inf, -np.inf], np.nan).dropna(subset=['ownership_fraction'])

ownership_by_type = holdings.groupby(['fdate', 'typecode'])['ownership_fraction'].mean().reset_index()

ownership_wide = ownership_by_type.pivot(index='fdate', columns='typecode', values='ownership_fraction').fillna(0)
ownership_wide['% Total'] = ownership_wide.sum(axis=1)

institution_type_mapping = {
    1: '% Bank trust depts.',
    2: '% Insurance',
    3: '% Mutual funds',
    4: '% Ind. inv. advisers',
    5: '% Unclassified'
}
ownership_wide.rename(columns=institution_type_mapping, inplace=True)

summary_stats = ownership_wide.agg(['mean', 'median', 'min', 'max']).T
summary_stats.columns = ['Mean', 'Median', 'Minimum', 'Maximum']
summary_stats = summary_stats.round(4).reset_index().rename(columns={'index': 'Variable'}) 

print(summary_stats) 
summary_stats.to_excel("summary_statisticsAA33.xlsx", index=False) 

               typecode    Mean  Median  Minimum  Maximum
0   % Bank trust depts.  0.0124  0.0107   0.0021   0.0507
1           % Insurance  0.0371  0.0276   0.0042   0.1526
2        % Mutual funds  0.0458  0.0373   0.0053   0.1646
3  % Ind. inv. advisers  0.0277  0.0280   0.0052   0.0504
4        % Unclassified  0.0289  0.0282   0.0084   0.0568
5               % Total  0.1519  0.1351   0.0313   0.3783


## Panel B - Share characteristics 

In [ ]:
### ORDINARY COMMON SHARES 

# CRSP Monthly Stock 
crsp_m = conn.raw_sql("""
    SELECT a.permno, a.date, a.ret, a.prc, a.shrout, a.vol, a.cfacpr, a.cfacshr, exchcd 
    FROM crsp.msf AS a
    LEFT JOIN crsp.msenames AS b
    ON a.permno = b.permno
    AND b.namedt <= a.date
    AND a.date <= b.nameendt
    WHERE a.date BETWEEN '01/01/1980' AND '12/31/2024'  -- Start earlier to calculate lagged measures
    AND b.shrcd BETWEEN 10 AND 11  -- Common stocks only
""", date_cols=['date'])

In [40]:
### EXTENSION 

# CRSP Monthly Stock 
crsp_m = conn.raw_sql("""
    SELECT a.permno, a.date, a.ret, a.prc, a.shrout, a.vol, a.cfacpr, a.cfacshr, exchcd 
    FROM crsp.msf AS a
    LEFT JOIN crsp.msenames AS b
    ON a.permno = b.permno
    AND b.namedt <= a.date
    AND a.date <= b.nameendt
    WHERE a.date BETWEEN '01/01/1980' AND '12/31/2024'  -- Start earlier to calculate lagged measures
    AND b.shrcd = 31  
""", date_cols=['date'])

In [ ]:
# Only keep stocks on NYSE (1), AMEX (2), or NASDAQ (3) ##### 
crsp_m = crsp_m[crsp_m['exchcd'].isin([1, 2, 3])]  

# CRSP Index (for Beta calculation)
# Use crsp.msi (Monthly Stock Index) instead of crsp.msti
crsp_index = conn.raw_sql("""
    SELECT date, vwretd  -- CRSP NYSE-AMEX value-weighted index return
    FROM crsp.msi
    WHERE date BETWEEN '01/01/1980' AND '12/31/2024' 
""", date_cols=['date'])

# Merge CRSP stock with index 
crsp_m = pd.merge(crsp_m, crsp_index, on='date', how='left') 

In [42]:
### GENERATE THE MEASURES 

# Adjust prices and compute market cap
crsp_m['p'] = crsp_m['prc'].abs() / crsp_m['cfacpr']
crsp_m['tso'] = crsp_m['shrout'] * crsp_m['cfacshr'] * 1e3  # Shares outstanding
crsp_m['me'] = crsp_m['p'] * crsp_m['tso'] / 1e6  # Market cap in $ millions

# Size (natural logarithm of market cap)
crsp_m = crsp_m[crsp_m['me'] > 0] 
crsp_m['size'] = np.log(crsp_m['me'])

# Price (natural logarithm of one plus price)
crsp_m['price'] = np.log(1 + crsp_m['p'])

# Turnover (natural logarithm of one plus monthly volume/shares outstanding)
crsp_m['turnover'] = np.log(1 + (crsp_m['vol']*1000 / crsp_m['tso']))

# Lag 6-Month Return
crsp_m['date'] = pd.to_datetime(crsp_m['date'])
crsp_m['lag_6m_return'] = crsp_m.groupby('permno')['ret'].rolling(window=6, min_periods=1).apply(lambda x: (1 + x).prod() - 1).reset_index(level=0, drop=True) 

In [ ]:
# Beta 

def compute_rolling_beta_firmwise(df):
    results = []
    for permno, g in df.groupby('permno'):
        g = g.sort_values('date').reset_index(drop=True)
        betas = []
        for i in range(len(g)):
            window = g.iloc[max(0, i-59):i+1]
            if len(window) >= 24 and window['vwretd'].std() > 0:
                cov = np.cov(window['ret'], window['vwretd'])[0, 1]
                var = np.var(window['vwretd'])
                beta = cov / var
            else:
                beta = np.nan
            betas.append(beta)
        g['beta'] = betas
        results.append(g[['permno', 'date', 'beta']])
    return pd.concat(results)

beta_df = compute_rolling_beta_firmwise(crsp_m[['permno', 'date', 'ret', 'vwretd']])
crsp_m = crsp_m.merge(beta_df, on=['permno', 'date'], how='left') 

In [30]:
# Standard Deviation (log of std of monthly returns over 24-60 months)
std_raw = crsp_m.groupby('permno')['ret'].rolling(window=60, min_periods=24).std().reset_index(level=0, drop=True)
crsp_m['std_dev'] = np.where(std_raw > 0, np.log(std_raw), np.nan)

# Firm-Specific Risk (log of 1 + firm-specific risk)
fsr_raw = crsp_m.groupby('permno')['ret'].rolling(window=3).apply(lambda x: ((x - x.mean())**2).sum()).reset_index(level=0, drop=True)
crsp_m['firm_specific_risk'] = np.where(fsr_raw >= 0, np.log(1 + fsr_raw), np.nan)

# Age (log of months since December 1972)
age_months = (crsp_m['date'] - pd.to_datetime('1972-12-01')).dt.days / 30
crsp_m['age'] = np.where(age_months > 0, np.log(age_months), np.nan) 

# Dividend Yield (log of 1 + average monthly dividend yield over the previous 12 months)
div_raw = crsp_m.groupby('permno')['ret'].rolling(window=12).apply(lambda x: x[x > 0].mean()).reset_index(level=0, drop=True)
crsp_m['div_yield'] = np.where(div_raw > 0, np.log(1 + div_raw), np.nan) 

# Number of Firms (count of unique permno in each quarter) 
crsp_m['qdate'] = crsp_m['date'] + pd.offsets.QuarterEnd(0)
num_firms = crsp_m.groupby('qdate')['permno'].nunique().reset_index(name='num_firms') 

In [32]:
crsp_m = crsp_m.dropna(subset=[
    'std_dev', 'firm_specific_risk', 'size', 'age',
    'div_yield', 'price', 'turnover', 'lag_6m_return' 
]) 

# WINSORIZE 
def fast_winsorize(series, lower=0.01, upper=0.99):
    q_low = series.quantile(lower)
    q_high = series.quantile(upper)
    return series.clip(q_low, q_high)

for col in [
    'std_dev', 'firm_specific_risk', 'size', 'age',
    'div_yield', 'price', 'turnover', 'lag_6m_return'
]:
    crsp_m[col] = fast_winsorize(crsp_m[col])

# Aggregate to Quarterly Data
crsp_q = crsp_m.groupby(['permno', 'qdate']).agg({
    'std_dev': 'last',
    'firm_specific_risk': 'last',
    'size': 'last',
    'age': 'last',
    'div_yield': 'last',
    'price': 'last',
    'turnover': 'last',
    'lag_6m_return': 'last'
}).reset_index()

crsp_q = pd.merge(crsp_q, num_firms, on='qdate', how='left') 

In [34]:
### SUMMARY STATISTICS 

summary_stats = crsp_q[[
    'std_dev', 'firm_specific_risk', 'size', 'age',
    'div_yield', 'price', 'turnover', 'lag_6m_return', 'num_firms'
]].agg(['mean', 'median', 'min', 'max']).T

summary_stats.columns = ['Mean', 'Median', 'Minimum', 'Maximum']
summary_stats = summary_stats.round(4).reset_index().rename(columns={'index': 'Variable'})

print(summary_stats) 
summary_stats.to_excel("summary_statisticsAAA1.xlsx", index=False) 

             Variable      Mean    Median  Minimum   Maximum
0             std_dev   -2.5573   -2.6472  -3.2700   -1.2461
1  firm_specific_risk    0.0181    0.0060   0.0001    0.2541
2                size    6.0910    6.2448   1.2387   10.5624
3                 age    5.9622    6.0352   4.8580    6.4457
4           div_yield    0.0673    0.0547   0.0198    0.2852
5               price    3.0416    2.9596   0.7538    6.3026
6            turnover    0.8617    0.7026   0.0226    4.3032
7       lag_6m_return    0.0495    0.0516  -0.6325    0.7473
8           num_firms  150.2609  162.0000  14.0000  191.0000


## TABLE 2. CORRELATION TABLE 

In [ ]:
ownership_cols = {
    0: '% Total',
    4: '% Ind. inv. advisers',
    1: '% Bank trust depts.',
    3: '% Mutual funds',
    2: '% Insurance',
    5: '% Unclassified'
}

ownership_fractions_renamed = ownership_wide.rename(columns=ownership_cols)

# Add permno and qdate to ownership_fractions before merging
if 'permno' not in ownership_wide.columns or 'fdate' not in ownership_wide.columns:
    firm_q_identifiers = holdings[['permno', 'fdate']].drop_duplicates()
    ownership_fractions = pd.concat([firm_q_identifiers.reset_index(drop=True), ownership_wide.reset_index(drop=True)], axis=1)

ownership_fractions.rename(columns={'fdate': 'qdate'}, inplace=True)

#Merge with firm-level share characteristics
full_df = pd.merge(ownership_fractions, crsp_q, on=['permno', 'qdate'], how='inner')

# Select final columns
ownership_vars = list(ownership_cols.values())
char_vars = ['std_dev', 'firm_specific_risk', 'size', 'age', 'div_yield', 'price', 'turnover', 'lag_6m_return']

# Correlation between institutional ownership and share characteristics
panel_a = pd.DataFrame(index=ownership_vars, columns=char_vars)
for o in ownership_vars:
    for c in char_vars:
        panel_a.loc[o, c] = full_df[o].corr(full_df[c])
panel_a = panel_a.astype(float).round(4)

# Panel B – Correlation among share characteristics
panel_b = full_df[char_vars].corr().round(4)

print("\nPanel A: Correlation between Institutional Ownership and Share Characteristics\n")
print(panel_a) 

print("\nPanel B: Correlation Among Share Characteristics\n")
print(panel_b) 

with pd.ExcelWriter("correlation_tablesA.xlsx") as writer:
    panel_a.to_excel(writer, sheet_name='Ownership vs. Characteristics')
    panel_b.to_excel(writer, sheet_name='Characteristic Correlations') 


Panel A: Correlation between Institutional Ownership and Share Characteristics

                      std_dev  firm_specific_risk    size     age  div_yield  \
% Total                0.0319              0.0128 -0.0175  0.1255    -0.1185   
% Ind. inv. advisers   0.0019              0.0146 -0.0192  0.1374    -0.1603   
% Bank trust depts.    0.0354              0.0853 -0.0666  0.1227    -0.1280   
% Mutual funds         0.0047              0.0807  0.0323  0.0940    -0.1098   
% Insurance            0.0836             -0.1011 -0.0448  0.0950    -0.0103   
% Unclassified         0.0232              0.0119  0.0014  0.1252    -0.1408   

                       price  turnover  lag_6m_return  
% Total              -0.1456   -0.2550        -0.0319  
% Ind. inv. advisers -0.1563   -0.2823        -0.0212  
% Bank trust depts.  -0.1469   -0.2506        -0.1071  
% Mutual funds       -0.2428   -0.2743         0.0099  
% Insurance           0.0018   -0.1152        -0.0393  
% Unclassified       -

In [60]:
with pd.ExcelWriter("correlation_panels1.xlsx") as writer:
    panel_a.to_excel(writer, sheet_name="Panel A")
    panel_b.to_excel(writer, sheet_name="Panel B") 

## TABLE 3. Fama-McBeth Cross-Sectional Regression 

In [ ]:
import statsmodels.api as sm
from scipy.stats import wilcoxon

char_vars = ['std_dev', 'firm_specific_risk', 'size', 'age',
             'div_yield', 'price', 'turnover', 'lag_6m_return']
target_var = '% Total' 

ownership_by_type = holdings.groupby(['permno', 'fdate', 'typecode'])['ownership_fraction'].mean().reset_index()
ownership_wide = ownership_by_type.pivot(index=['permno', 'fdate'], columns='typecode', values='ownership_fraction').fillna(0)
ownership_wide = ownership_wide.reset_index().rename(columns={'fdate': 'qdate'})

type_map = {
    1: '% Bank trust depts.',
    2: '% Insurance',
    3: '% Mutual funds',
    4: '% Ind. inv. advisers',
    5: '% Unclassified'
}
ownership_wide = ownership_wide.rename(columns=type_map)
ownership_wide['% Total'] = ownership_wide[[v for v in type_map.values()]].sum(axis=1)

# Merge with share characteristics
reg_df = pd.merge(ownership_wide, crsp_q, on=['permno', 'qdate'], how='inner')
reg_df = reg_df.dropna(subset=[target_var] + char_vars)

# Run regressions quarter-by-quarter
coef_list = []
tstat_list = []
se_list = []

# Panel B
periods = reg_df['qdate'].sort_values().unique()
split = len(periods) // 2
period1 = periods[:split]
period2 = periods[split:]
coef_p1, coef_p2 = [], []
tstat_p1, tstat_p2 = [], []
se_p1, se_p2 = [], []

for date, group in reg_df.groupby('qdate'):
    df = group.copy()
    df[char_vars + [target_var]] = df[char_vars + [target_var]].apply(lambda x: (x - x.mean()) / x.std())
    X = sm.add_constant(df[char_vars])
    y = df[target_var]
    data = pd.concat([X, y], axis=1).replace([np.inf, -np.inf], np.nan).dropna()
    X_clean = data[X.columns]
    y_clean = data[target_var]

    if len(X_clean) >= len(char_vars) + 2:
        model = sm.OLS(y_clean, X_clean).fit(cov_type='HC0')
        betas = model.params[1:]
        ses = model.bse[1:]
        tstats = model.tvalues[1:]

        coef_list.append(betas)
        se_list.append(ses)
        tstat_list.append(tstats)

        if date in period1:
            coef_p1.append(betas)
            se_p1.append(ses)
            tstat_p1.append(tstats)
        else:
            coef_p2.append(betas)
            se_p2.append(ses)
            tstat_p2.append(tstats)

coef_df = pd.DataFrame(coef_list)
se_df = pd.DataFrame(se_list)
tstat_df = pd.DataFrame(tstat_list)

panel_a = pd.DataFrame({
    'Avg Coef': coef_df.mean(),
    'Std Error': se_df.mean(),
    't-Stat': tstat_df.mean()
}).round(4)


coef_p1_df = pd.DataFrame(coef_p1).mean().round(4)
se_p1_df = pd.DataFrame(se_p1).mean().round(4)
tstat_p1_df = pd.DataFrame(tstat_p1).mean().round(4)

coef_p2_df = pd.DataFrame(coef_p2).mean().round(4)
se_p2_df = pd.DataFrame(se_p2).mean().round(4)
tstat_p2_df = pd.DataFrame(tstat_p2).mean().round(4)

panel_b = pd.DataFrame({
    'Coef (P1)': coef_p1_df,
    'SE (P1)': se_p1_df,
    't-Stat (P1)': tstat_p1_df,
    'Coef (P2)': coef_p2_df,
    'SE (P2)': se_p2_df,
    't-Stat (P2)': tstat_p2_df
})


wilcoxon_results = []
for var in char_vars:
    try:
        stat, p = wilcoxon(pd.DataFrame(coef_p1)[var], pd.DataFrame(coef_p2)[var])
        wilcoxon_results.append((stat, p))
    except:
        wilcoxon_results.append((np.nan, np.nan))

wilcoxon_df = pd.DataFrame(wilcoxon_results, columns=['Wilcoxon z', 'p'], index=char_vars).round(3)


print("\n=== Panel A: Full Sample ===")
print(panel_a)

print("\n=== Panel B: Subsample Means ===")
print(panel_b)

print("\n=== Wilcoxon Rank-Sum Test Results ===")
print(wilcoxon_df)

with pd.ExcelWriter("regression_resultsAA.xlsx") as writer:
    panel_a.to_excel(writer, sheet_name='Panel A - Full Sample')
    panel_b.to_excel(writer, sheet_name='Panel B - Subsamples')
    wilcoxon_df.to_excel(writer, sheet_name='Wilcoxon Test') 


=== Panel A: Full Sample ===
                    Avg Coef  Std Error  t-Stat
std_dev               0.0039     0.0196  0.0486
firm_specific_risk   -0.0003     0.0163 -0.3673
size                 -0.0308     0.0275 -1.0818
age                   0.0024     0.0080  0.9947
div_yield            -0.0015     0.0242 -0.2847
price                 0.0509     0.0346  1.3989
turnover              0.1202     0.0720  1.6499
lag_6m_return        -0.0067     0.0168 -0.5922

=== Panel B: Subsample Means ===
                    Coef (P1)  SE (P1)  t-Stat (P1)  Coef (P2)  SE (P2)  \
std_dev                0.0122   0.0215       0.3253    -0.0047   0.0177   
firm_specific_risk    -0.0022   0.0156      -0.4504     0.0016   0.0170   
size                  -0.0139   0.0234      -0.6588    -0.0482   0.0318   
age                    0.0035   0.0059       0.8912     0.0013   0.0101   
div_yield             -0.0071   0.0222      -0.2778     0.0043   0.0262   
price                  0.0297   0.0235       1.2195   